In [ ]:
!pip uninstall tensorflow

In [ ]:
!pip install tensorflow==1.15 # 1.15버전 Tensorflow 설치

# Tensor Manipulation

In [4]:
import tensorflow as tf
import numpy as np
import pprint # 임의의 파이썬 데이터 구조를 인터프리터의 입력으로 사용할 수 있는 형태를 지원하는 라이브러리
tf.set_random_seed(777)

pp = pprint.PrettyPrinter(indent = 4) # indent의 경우 들여쓰기를 위한 칸 수

# tf.Session()을 사용하는 경우 자동으로 세션을 할당하지 않기 때문에 주로 with문과 함께 쓰인다.
sess = tf.InteractiveSession() # 이는 자동으로 터미널에 default session을 할당한다.

### Simple Arrary

In [5]:
t = np.array([0., 1., 2., 3., 4., 5., 6.])
pp.pprint(t)
print(t.ndim) # rank
print(t.shape) # shape
print(t[0], t[1], t[-1])
print(t[2:5], t[4:-1])
print(t[:2], t[3:])

array([0., 1., 2., 3., 4., 5., 6.])
1
(7,)
0.0 1.0 6.0
[2. 3. 4.] [4. 5.]
[0. 1.] [3. 4. 5. 6.]


### 2D Array

In [6]:
t = np.array([[1., 2., 3.], [4., 5., 6.], [7., 8., 9.], [10., 11., 12.]])
pp.pprint(t)
print(t.ndim) # rank
print(t.shape) # shape

array([[ 1.,  2.,  3.],
       [ 4.,  5.,  6.],
       [ 7.,  8.,  9.],
       [10., 11., 12.]])
2
(4, 3)


### Shape, Rank, Axis
- 축의 경우 axis = -1을  하면 제일 안쪽 축을 의미

In [7]:
t = tf.constant([1,2,3,4])
tf.shape(t).eval()

array([4], dtype=int32)

In [8]:
t = tf.constant([[1,2],
                 [3,4]])
tf.shape(t).eval()

array([2, 2], dtype=int32)

In [9]:
# n 차원을 확인할 때는 가장 안쪽의 차원부터 확인하면 쉽게 차원을 확인할 수 있음
t = tf.constant([[[[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12]],[[13, 14, 15, 16], [17, 18, 19, 20], [21, 22, 23, 24]]]]) 
tf.shape(t).eval()

array([1, 2, 3, 4], dtype=int32)

In [10]:
[
    [
        [
            [1,2,3,4], 
            [5,6,7,8],
            [9,10,11,12]
        ],
        [
            [13,14,15,16],
            [17,18,19,20], 
            [21,22,23,24]
        ]
    ]
]

[[[[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12]],
  [[13, 14, 15, 16], [17, 18, 19, 20], [21, 22, 23, 24]]]]

### Matmul VS multiply

In [11]:
# [3, 3,] * [[2],
#            [2]]
matrix1 = tf.constant([[3., 3.]])
matrix2 = tf.constant([[2.],[2.]])
tf.matmul(matrix1, matrix2).eval()

array([[12.]], dtype=float32)

In [12]:
(matrix1*matrix2).eval() # 이는 일반적인 행렬 곱셈이 아니라 Broadcasting되어 원소끼리의 곱이 계산된 것

array([[6., 6.],
       [6., 6.]], dtype=float32)

### Watch out broadcasting

In [13]:
# Broadcasting은 Tensor의 차원을 맞춰주는 역할을 함
# [3, 3] + [[2], 였던 것이 [[3, 3], + [[2, 2],  로 변경됨
#           [2]]            [3, 3]]    [2, 2]] 
matrix1 = tf.constant([[3., 3.]])
matrix2 = tf.constant([[2.],[2.]])
(matrix1+matrix2).eval()

array([[5., 5.],
       [5., 5.]], dtype=float32)

In [14]:
matrix1 = tf.constant([[3., 3.]])
matrix2 = tf.constant([[2., 2.]])
(matrix1+matrix2).eval()

array([[5., 5.]], dtype=float32)

### Random values for variable initialization

In [15]:
tf.random_normal([3]).eval() # 정규분포로 부터 받은 Tensor의 차원만큼 난수를 반환한다.

array([ 1.211047  ,  0.5397847 , -0.41038424], dtype=float32)

In [16]:
tf.random_uniform([2]).eval() # 균등 분포로부터 받은 Tensor의 차원만큼 난수를 반환한다. 
# 이때 [minval, maxval) 사이의 균등한 난수 값을 반환

array([0.77554226, 0.9517144 ], dtype=float32)

In [17]:
tf.random_uniform([2, 3]).eval()

array([[0.21917546, 0.21820867, 0.22875011],
       [0.88537264, 0.01493788, 0.22606587]], dtype=float32)

### Reduced Mean/Sum

In [18]:
tf.reduce_mean([1, 2], axis = 0).eval()

1

In [19]:
x = [[1., 2.],
     [3., 4.]]

tf.reduce_mean(x).eval() # 축의 설정이 없으면 전체에 대한 값을 구함

2.5

In [20]:
tf.reduce_mean(x, axis=0).eval()

array([2., 3.], dtype=float32)

In [21]:
tf.reduce_mean(x, axis=1).eval()

array([1.5, 3.5], dtype=float32)

In [22]:
tf.reduce_mean(x, axis=-1).eval()

array([1.5, 3.5], dtype=float32)

In [23]:
tf.reduce_sum(x).eval()

10.0

In [24]:
tf.reduce_sum(x, axis=0).eval()

array([4., 6.], dtype=float32)

In [25]:
tf.reduce_sum(x, axis=-1).eval()

array([3., 7.], dtype=float32)

In [26]:
tf.reduce_mean(tf.reduce_sum(x, axis=-1)).eval()

5.0

### Argmax with axis

In [27]:
# argmax는 축을 기준으로 큰 값의 위치(index)를 구해줌
x = [[0, 1, 2],
     [2, 1, 0]]
tf.argmax(x, axis=0).eval()

array([1, 0, 0])

In [28]:
tf.argmax(x, axis=1).eval()

array([2, 0])

In [29]:
tf.argmax(x, axis=-1).eval()

array([2, 0])

### Reshape, Squeeze, expand_dims

In [30]:
# reshape에서 가장 마지막 축은 통상적으로 변경하지 않고 나머지를 조정한다.
t = np.array([[[0, 1, 2], 
               [3, 4, 5]],
              
              [[6, 7, 8], 
               [9, 10, 11]]])
t.shape

(2, 2, 3)

In [31]:
tf.reshape(t, shape=[-1, 3]).eval()

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11]])

In [32]:
tf.reshape(t, shape=[-1, 1, 3]).eval()

array([[[ 0,  1,  2]],

       [[ 3,  4,  5]],

       [[ 6,  7,  8]],

       [[ 9, 10, 11]]])

In [33]:
tf.squeeze([[[0]], [[1]], [[2]]]).eval() # Tensor에서 크기가 1인 차원을 없애줌

array([0, 1, 2], dtype=int32)

In [34]:
# 이는 크기가 1인 차원을 삽입한다.
tf.expand_dims([0, 1, 2], 1).eval() # -1-input.dims() <= dim <= input.dims()안에서 확장가능

array([[0],
       [1],
       [2]], dtype=int32)

### One hot

In [35]:
# One hot encoding을 수행하면 차원이 1개 늘어남
tf.one_hot([[0], [1], [2], [0]], depth=3).eval() # depth인자는 encoding의 길이를 결정, 따라서 이는 데이터 Label의 수만큼 지정해주면 됨

array([[[1., 0., 0.]],

       [[0., 1., 0.]],

       [[0., 0., 1.]],

       [[1., 0., 0.]]], dtype=float32)

In [36]:
t = tf.one_hot([[0], [1], [2], [0]], depth=3)
tf.reshape(t, shape=[-1, 3]).eval()

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.]], dtype=float32)

### Casting

In [37]:
# casting은 데이터를 원하는 type으로 변경함
tf.cast([1.8, 2.2, 3.3, 4.9], tf.int32).eval()

array([1, 2, 3, 4], dtype=int32)

In [38]:
tf.cast([True, False, 1 == 1, 0 == 1], tf.int32).eval()

array([1, 0, 1, 0], dtype=int32)

### Stack

In [39]:
# stack은 축을 기준으로 stack을 한다. 축을 설정하지 않으면 변수의 순서대로 stack에 쌓인다.
x = [1, 4]
y = [2, 5]
z = [3, 6]

# Pack along first dim.
tf.stack([x, y, z]).eval()

array([[1, 4],
       [2, 5],
       [3, 6]], dtype=int32)

In [40]:
tf.stack([x, y, z], axis=1).eval()

array([[1, 2, 3],
       [4, 5, 6]], dtype=int32)

### Ones like and Zeros like

In [41]:
# 이는 인자로 받은 데이터의 차원과 똑같은 크기에 1 or 0을 삽입하여 생성한다.
x = [[0, 1, 2],
     [2, 1, 0]]

tf.ones_like(x).eval()

array([[1, 1, 1],
       [1, 1, 1]], dtype=int32)

In [42]:
tf.zeros_like(x).eval()

array([[0, 0, 0],
       [0, 0, 0]], dtype=int32)

### Zip

In [43]:
for x, y in zip([1, 2, 3], [4, 5, 6]):
    print(x, y)

1 4
2 5
3 6


In [44]:
for x, y, z in zip([1, 2, 3], [4, 5, 6], [7, 8, 9]):
    print(x, y, z)

1 4 7
2 5 8
3 6 9


### Transpose

In [45]:
t = np.array([[[0, 1, 2], [3, 4, 5]], [[6, 7, 8], [9, 10, 11]]])
pp.pprint(t.shape)
pp.pprint(t)

(2, 2, 3)
array([[[ 0,  1,  2],
        [ 3,  4,  5]],

       [[ 6,  7,  8],
        [ 9, 10, 11]]])


In [46]:
t1 = tf.transpose(t, [1, 0, 2]) # perm은 해당 Tensor의 차원만큼 생성하여 index 위치를 변경하면 된다. 이는 Permutation을 사용한다.
pp.pprint(sess.run(t1).shape)
pp.pprint(sess.run(t1))

(2, 2, 3)
array([[[ 0,  1,  2],
        [ 6,  7,  8]],

       [[ 3,  4,  5],
        [ 9, 10, 11]]])


In [47]:
t2 = tf.transpose(t, [1, 2, 0])
pp.pprint(sess.run(t2).shape)
pp.pprint(sess.run(t2))

(2, 3, 2)
array([[[ 0,  6],
        [ 1,  7],
        [ 2,  8]],

       [[ 3,  9],
        [ 4, 10],
        [ 5, 11]]])


In [48]:
t = tf.transpose(t2, [2, 0, 1])
pp.pprint(sess.run(t).shape)
pp.pprint(sess.run(t))

(2, 2, 3)
array([[[ 0,  1,  2],
        [ 3,  4,  5]],

       [[ 6,  7,  8],
        [ 9, 10, 11]]])
